Read the training data as a pandas dataframe.

In [1]:
import pandas as pd

train_file = 'task1_trainset.csv'
df = pd.read_csv(train_file) 
df

,Id,Title,Abstract,Authors,Categories,Created Date,Task 1
0,D00001,A Brain-Inspired Trust Management Model to Ass...,Rapid popularity of Internet of Things (IoT) a...,Mahmud/Kaiser/Rahman/Rahman/Shabut/Al-Mamun/Hu...,cs.CR/cs.AI/q-bio.NC,2018-01-11,BACKGROUND OBJECTIVES METHODS METHODS RESULTS ...
1,D00002,On Efficient Computation of Shortest Dubins Pa...,"In this paper, we address the problem of compu...",Sadeghi/Smith,cs.SY/cs.RO/math.OC,2016-09-21,OBJECTIVES OTHERS METHODS/RESULTS RESULTS RESULTS
2,D00003,Data-driven Upsampling of Point Clouds,High quality upsampling of sparse 3D point clo...,Zhang/Jiang/Yang/Yamakawa/Shimada/Kara,cs.CV,2018-07-07,BACKGROUND OBJECTIVES METHODS METHODS METHODS ...
3,D00004,Accessibility or Usability of InteractSE? A He...,Internet is the main source of information now...,Aqle/Khowaja/Al-Thani,cs.HC,2018-08-29,BACKGROUND BACKGROUND BACKGROUND OBJECTIVES OB...
4,D00005,Spatio-Temporal Facial Expression Recognition ...,Automated Facial Expression Recognition (FER) ...,Hasani/Mahoor,cs.CV,2017-03-20,BACKGROUND BACKGROUND BACKGROUND BACKGROUND ME...
5,D00006,Continuous Semantic Topic Embedding Model Usin...,This paper proposes the continuous semantic to...,Jung/Choi,stat.ML/cs.CL/cs.LG,2017-11-24,OBJECTIVES/METHODS METHODS CONCLUSIONS RESULTS...
6,D00007,Beyond Shared Hierarchies: Deep Multitask Lear...,Existing deep multitask learning (MTL) approac...,Meyerson/Miikkulainen,cs.LG/cs.AI/stat.ML,2017-10-31,BACKGROUND BACKGROUND/OBJECTIVES OBJECTIVES/ME...
7,D00008,Using Scalp Electrical Biosignals to Control a...,In this paper we explore the use of electrical...,George/Lotte/Abad/Lécuyer,cs.OH,2011-11-08,OBJECTIVES METHODS METHODS METHODS METHODS RES...
8,D00009,Robustness from structure: Inference with hier...,How spiking networks are able to perform proba...,Petrovici/Schroeder/Breitwieser/Grübl/Schemmel...,q-bio.NC/cs.NE/stat.ML,2017-03-12,BACKGROUND/OBJECTIVES BACKGROUND BACKGROUND/ME...
9,D00010,Statistical Mechanical Analysis of Low-Density...,Low-density parity-check (LDPC) codes on symme...,Mori/Tanaka,cs.IT/math.IT,2011-10-10,BACKGROUND OBJECTIVES RESULTS/CONCLUSIONS METHODS


According to the description:  


- Abstract
    Abstract of the paper. Each sentence is be seperated by **$$$**.

      

- Task 1:
    Multilabels of each sentence in the paper. Each multilabel is be separated by **space** and each label is be seperated by **/**. (每個句子的分類以 **空格** 分開，同個句子多個分類以 **/** 分開) 


Goal : tag each sentence in abstract with a label/multilabel.  
Let's query how it was tagged in training data.

In [2]:
df['Task 1'][:10]

0    BACKGROUND OBJECTIVES METHODS METHODS RESULTS ...
1    OBJECTIVES OTHERS METHODS/RESULTS RESULTS RESULTS
2    BACKGROUND OBJECTIVES METHODS METHODS METHODS ...
3    BACKGROUND BACKGROUND BACKGROUND OBJECTIVES OB...
4    BACKGROUND BACKGROUND BACKGROUND BACKGROUND ME...
5    OBJECTIVES/METHODS METHODS CONCLUSIONS RESULTS...
6    BACKGROUND BACKGROUND/OBJECTIVES OBJECTIVES/ME...
7    OBJECTIVES METHODS METHODS METHODS METHODS RES...
8    BACKGROUND/OBJECTIVES BACKGROUND BACKGROUND/ME...
9    BACKGROUND OBJECTIVES RESULTS/CONCLUSIONS METHODS
Name: Task 1, dtype: object

In [3]:
[df['Task 1'][i] for i in range(0,11)]

['BACKGROUND OBJECTIVES METHODS METHODS RESULTS CONCLUSIONS',
 'OBJECTIVES OTHERS METHODS/RESULTS RESULTS RESULTS',
 'BACKGROUND OBJECTIVES METHODS METHODS METHODS METHODS METHODS RESULTS/CONCLUSIONS',
 'BACKGROUND BACKGROUND BACKGROUND OBJECTIVES OBJECTIVES METHODS METHODS METHODS METHODS METHODS RESULTS RESULTS RESULTS RESULTS RESULTS CONCLUSIONS',
 'BACKGROUND BACKGROUND BACKGROUND BACKGROUND METHODS METHODS METHODS METHODS METHODS METHODS OBJECTIVES/RESULTS/CONCLUSIONS',
 'OBJECTIVES/METHODS METHODS CONCLUSIONS RESULTS RESULTS',
 'BACKGROUND BACKGROUND/OBJECTIVES OBJECTIVES/METHODS METHODS/RESULTS RESULTS CONCLUSIONS',
 'OBJECTIVES METHODS METHODS METHODS METHODS RESULTS RESULTS RESULTS RESULTS RESULTS',
 'BACKGROUND/OBJECTIVES BACKGROUND BACKGROUND/METHODS OBJECTIVES/METHODS/RESULTS OBJECTIVES/METHODS/RESULTS/CONCLUSIONS',
 'BACKGROUND OBJECTIVES RESULTS/CONCLUSIONS METHODS',
 'BACKGROUND BACKGROUND BACKGROUND BACKGROUND OBJECTIVES METHODS METHODS/RESULTS RESULTS/CONCLUSIONS']

In [4]:
[df['Abstract'][i] for i in range(0,11)]

['Rapid popularity of Internet of Things (IoT) and cloud computing permits neuroscientists to collect multilevel and multichannel brain data to better understand brain functions, diagnose diseases, and devise treatments.$$$To ensure secure and reliable data communication between end-to-end (E2E) devices supported by current IoT and cloud infrastructure, trust management is needed at the IoT and user ends.$$$This paper introduces a Neuro-Fuzzy based Brain-inspired trust management model (TMM) to secure IoT devices and relay nodes, and to ensure data reliability.$$$The proposed TMM utilizes node behavioral trust and data trust estimated using Adaptive Neuro-Fuzzy Inference System and weighted-additive methods respectively to assess the nodes trustworthiness.$$$In contrast to the existing fuzzy based TMMs, the NS2 simulation results confirm the robustness and accuracy of the proposed TMM in identifying malicious nodes in the communication network.$$$With the growing usage of cloud based I

In [5]:
print(df['Task 1'][133])
df['Abstract'][133]

BACKGROUND BACKGROUND BACKGROUND BACKGROUND/OBJECTIVES METHODS OBJECTIVES/METHODS METHODS METHODS RESULTS/CONCLUSIONS METHODS OTHERS


'The Internet of Things (IoT) being a promising technology of the future is expected to connect billions of devices.$$$The increased number of communication is expected to generate mountains of data and the security of data can be a threat.$$$The devices in the architecture are essentially smaller in size and low powered.$$$Conventional encryption algorithms are generally computationally expensive due to their complexity and requires many rounds to encrypt, essentially wasting the constrained energy of the gadgets.$$$Less complex algorithm, however, may compromise the desired integrity.$$$In this paper we propose a lightweight encryption algorithm named as Secure IoT (SIT).$$$It is a 64-bit block cipher and requires 64-bit key to encrypt the data.$$$The architecture of the algorithm is a mixture of feistel and a uniform substitution-permutation network.$$$Simulations result shows the algorithm provides substantial security in just five encryption rounds.$$$The hardware implementation o

In [8]:
for sent, tag in zip(df['Abstract'][133].split('$$$'), df['Task 1'][133].split(' ')):
    print('%s=>%s'%(sent, tag))

The Internet of Things (IoT) being a promising technology of the future is expected to connect billions of devices.=>BACKGROUND
The increased number of communication is expected to generate mountains of data and the security of data can be a threat.=>BACKGROUND
The devices in the architecture are essentially smaller in size and low powered.=>BACKGROUND
Conventional encryption algorithms are generally computationally expensive due to their complexity and requires many rounds to encrypt, essentially wasting the constrained energy of the gadgets.=>BACKGROUND/OBJECTIVES
Less complex algorithm, however, may compromise the desired integrity.=>METHODS
In this paper we propose a lightweight encryption algorithm named as Secure IoT (SIT).=>OBJECTIVES/METHODS
It is a 64-bit block cipher and requires 64-bit key to encrypt the data.=>METHODS
The architecture of the algorithm is a mixture of feistel and a uniform substitution-permutation network.=>METHODS
Simulations result shows the algorithm prov

* create another feature that tag every sentence with the label

In [2]:
df['tag_sent'] = df[['Abstract','Task 1']].apply(lambda x: list(zip(x['Abstract'].split('$$$'), x['Task 1'].split(' '))), axis=1)

In [4]:
df

,Id,Title,Abstract,Authors,Categories,Created Date,Task 1,tag_sent
0,D00001,A Brain-Inspired Trust Management Model to Ass...,Rapid popularity of Internet of Things (IoT) a...,Mahmud/Kaiser/Rahman/Rahman/Shabut/Al-Mamun/Hu...,cs.CR/cs.AI/q-bio.NC,2018-01-11,BACKGROUND OBJECTIVES METHODS METHODS RESULTS ...,[(Rapid popularity of Internet of Things (IoT)...
1,D00002,On Efficient Computation of Shortest Dubins Pa...,"In this paper, we address the problem of compu...",Sadeghi/Smith,cs.SY/cs.RO/math.OC,2016-09-21,OBJECTIVES OTHERS METHODS/RESULTS RESULTS RESULTS,"[(In this paper, we address the problem of com..."
2,D00003,Data-driven Upsampling of Point Clouds,High quality upsampling of sparse 3D point clo...,Zhang/Jiang/Yang/Yamakawa/Shimada/Kara,cs.CV,2018-07-07,BACKGROUND OBJECTIVES METHODS METHODS METHODS ...,[(High quality upsampling of sparse 3D point c...
3,D00004,Accessibility or Usability of InteractSE? A He...,Internet is the main source of information now...,Aqle/Khowaja/Al-Thani,cs.HC,2018-08-29,BACKGROUND BACKGROUND BACKGROUND OBJECTIVES OB...,[(Internet is the main source of information n...
4,D00005,Spatio-Temporal Facial Expression Recognition ...,Automated Facial Expression Recognition (FER) ...,Hasani/Mahoor,cs.CV,2017-03-20,BACKGROUND BACKGROUND BACKGROUND BACKGROUND ME...,[(Automated Facial Expression Recognition (FER...
5,D00006,Continuous Semantic Topic Embedding Model Usin...,This paper proposes the continuous semantic to...,Jung/Choi,stat.ML/cs.CL/cs.LG,2017-11-24,OBJECTIVES/METHODS METHODS CONCLUSIONS RESULTS...,[(This paper proposes the continuous semantic ...
6,D00007,Beyond Shared Hierarchies: Deep Multitask Lear...,Existing deep multitask learning (MTL) approac...,Meyerson/Miikkulainen,cs.LG/cs.AI/stat.ML,2017-10-31,BACKGROUND BACKGROUND/OBJECTIVES OBJECTIVES/ME...,[(Existing deep multitask learning (MTL) appro...
7,D00008,Using Scalp Electrical Biosignals to Control a...,In this paper we explore the use of electrical...,George/Lotte/Abad/Lécuyer,cs.OH,2011-11-08,OBJECTIVES METHODS METHODS METHODS METHODS RES...,[(In this paper we explore the use of electric...
8,D00009,Robustness from structure: Inference with hier...,How spiking networks are able to perform proba...,Petrovici/Schroeder/Breitwieser/Grübl/Schemmel...,q-bio.NC/cs.NE/stat.ML,2017-03-12,BACKGROUND/OBJECTIVES BACKGROUND BACKGROUND/ME...,[(How spiking networks are able to perform pro...
9,D00010,Statistical Mechanical Analysis of Low-Density...,Low-density parity-check (LDPC) codes on symme...,Mori/Tanaka,cs.IT/math.IT,2011-10-10,BACKGROUND OBJECTIVES RESULTS/CONCLUSIONS METHODS,[(Low-density parity-check (LDPC) codes on sym...


In [15]:
df['tag_sent'][::10][:10]

0     [(Rapid popularity of Internet of Things (IoT)...
10    [(Mobile agent networks, such as multi-UAV sys...
20    [(Centrality is an important notion in complex...
30    [(We consider the problem of extracting entrop...
40    [(A mobile robot deployed for remote inspectio...
50    [(In wind farms, wake interaction leads to los...
60    [(Although nonstationary data are more common ...
70    [(We consider a compressed sensing problem in ...
80    [(Solar forecasting accuracy is affected by we...
90    [(We introduce a hybrid (discrete--continuous)...
Name: tag_sent, dtype: object

In [16]:
[pair for pair in df['tag_sent'][::10][:10]]

[[('Rapid popularity of Internet of Things (IoT) and cloud computing permits neuroscientists to collect multilevel and multichannel brain data to better understand brain functions, diagnose diseases, and devise treatments.',
   'BACKGROUND'),
  ('To ensure secure and reliable data communication between end-to-end (E2E) devices supported by current IoT and cloud infrastructure, trust management is needed at the IoT and user ends.',
   'OBJECTIVES'),
  ('This paper introduces a Neuro-Fuzzy based Brain-inspired trust management model (TMM) to secure IoT devices and relay nodes, and to ensure data reliability.',
   'METHODS'),
  ('The proposed TMM utilizes node behavioral trust and data trust estimated using Adaptive Neuro-Fuzzy Inference System and weighted-additive methods respectively to assess the nodes trustworthiness.',
   'METHODS'),
  ('In contrast to the existing fuzzy based TMMs, the NS2 simulation results confirm the robustness and accuracy of the proposed TMM in identifying mal

In [3]:
df['tag_sent'][134]

[('Social media for news consumption is becoming increasingly popular due to its easy access, fast dissemination, and low cost.',
  'BACKGROUND'),
 ('However, social media also enable the wide propagation of "fake news", i.e., news with intentionally false information.',
  'BACKGROUND'),
 ('Fake news on social media poses significant negative societal effects, and also presents unique challenges.',
  'BACKGROUND'),
 ('To tackle the challenges, many existing works exploit various features, from a network perspective, to detect and mitigate fake news.',
  'OBJECTIVES'),
 ('In essence, news dissemination ecosystem involves three dimensions on social media, i.e., a content dimension, a social dimension, and a temporal dimension.',
  'OBJECTIVES'),
 ('In this chapter, we will review network properties for studying fake news, introduce popular network types and how these networks can be used to detect and mitigation fake news on social media.',
  'METHODS')]

In [ ]:
# grep every sents from each label
# y/n for each label
# punc e.g. ""-> background, &, = -> method
# sent_len
# dict with {label:[sents_under_that_label]}